# City Planner Application

The eventual goal is to have a GUI-based application that can produce a variety of plots that the climate impacts community would find useful.   One of these plots is "Number of Days with Temperature over X Degrees" for some GPS location and time range.

The application should eventually allow choices for:
  * Data source (LENS, CMIP6, NA-CORDEX, etc.)
  * Climate Application  (Extreme Heat Events, Growing Season, Extreme Precipitation Events, etc.)
  * GPS location if applicable
  * Time range
  * Units (Fahrenheit vs. Celsius, etc.)
  * Plot type (Scatter plots vs Box plots, etc.)

In [ ]:
# Display output of plots directly in Notebook
%matplotlib inline

In [ ]:
import xarray as xr
import numpy as np
import intake
from tqdm.auto import tqdm
import shutil 
import os
from functools import reduce
import pprint
import json
from operator import mul
import random
import yaml
from distributed.utils import format_bytes
import dask

In [ ]:
cluster.close()

In [ ]:
# Create cluster
from dask_gateway import Gateway
from dask.distributed import Client
gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=2, maximum=100)
# Connect to cluster
client = Client(cluster)
# Display cluster dashboard URL
cluster

### Starting Point:  Create static temperature Plot for Boulder, Colorado from CESM LENS Data

In [ ]:

# Load the data catalog.  Use a field separator character for key generation that does not conflict with strings in the catalog.
intakeEsmUrl = 'https://ncar-cesm-lens.s3-us-west-2.amazonaws.com/catalogs/aws-cesm1-le.json'
col = intake.open_esm_datastore(intakeEsmUrl)

# List a few catalog entries for an idea of the catalog structure.  
# Note: in general, each catalog has different columns.
col.df.head()

In [ ]:
# Discover what is available for Daily Temperature at Reference Height (TREFHT)
subset = col.search(variable='TREFHT')
subset.unique(columns=['experiment', 'component','frequency'])

In [ ]:
# Let's limit the date range to [1850, 2100] by choosing a subset of experiments, and choose the "daily" output stream.
subset = col.search(variable='TREFHT', experiment=['20C','RCP85'], frequency='daily')
subset.df

### Create Xarray Objects from this data subset

Note that the operation that merges data into Xarray Objects is relatively expensive, so we create a Dask cluster to process it quickly.

In [ ]:
# Load catalog entries for subset into a dictionary of xarray datasets
dsets = subset.to_dataset_dict(zarr_kwargs={"consolidated": True}, storage_options={"anon": True})
print("\nDataset dictionary keys:\n", dsets.keys())

In [ ]:
daily_temps_1850_2005 = dsets['atm.20C.daily']
daily_temps_2006_2100 = dsets['atm.RCP85.daily']

In [ ]:
(boulder_lat, boulder_lon) = (40.01, 244.8)

In [ ]:
boulder_temps1 = daily_temps_1850_2005.sel(lat=boulder_lat, lon=boulder_lon, method='nearest')
boulder_temps2 = daily_temps_2006_2100.sel(lat=boulder_lat, lon=boulder_lon, method='nearest')

In [ ]:
boulder_temps_1990 = boulder_temps1.sel(time=slice('1990-01-01', '1991-01-01'))
boulder_temps_2090 = boulder_temps2.sel(time=slice('2090-01-01', '2091-01-01'))

In [ ]:
boulder_temps_1990

In [ ]:
boulder_temps_2090

In [ ]:
df_1990 = boulder_temps_1990.TREFHT.to_dataframe()

In [ ]:
df_2090 = boulder_temps_2090.TREFHT.to_dataframe()

In [ ]:
df_1990.to_csv('boulder_1990.csv',index=True)


In [ ]:
df_2090.to_csv('boulder_2090.csv',index=True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
times = boulder_temps_1990.time.data
times[:3]

In [ ]:
time_indexes = np.arange(1,times.size + 1)

In [ ]:
temps = boulder_temps_1990.TREFHT.data.transpose()
temps

In [ ]:
ax = plt.axes()
ax.plot(time_indexes, temps)

In [ ]:
ax = plt.axes()
ax.plot(time_indexes, temps)

In [ ]:
cluster.close()

### Test NA-CORDEX Catalog Behavior Here

In [ ]:
import intake

In [ ]:
cordex_path = '/glade/u/home/bonnland/GitRepos/PullRequests/intake-esm-datastore-sethmcg/catalogs/glade-na-cordex.json'

In [ ]:
col = intake.open_esm_datastore(cordex_path)

In [ ]:
col

## Explore Zarr files

In [ ]:
import xarray as xr


In [ ]:
ds = xr.open_zarr("/glade/scratch/bonnland/lens-aws/ocn/monthly/cesmLE-CTRL-SFWF.zarr", consolidated=True)

In [ ]:
ds